# Notebook for create multiple agent for hypothesis

1 agent for summarize imformation from the gene list (similar to the one we used for the llm eval paper)

1 agent fact-check the find references.

1 agent to create a pathway knowledge graph (following the INDRA terms)

1 agent to create hypothesis statement summarizing known information 

    * additional agent to check novelty (that should be context specific), 
        * Is the hypothesis already supported in the form of paper? 
        * If no, likly novel 
        * If yes, Is it in the same context? if no, also novel (more plausible hypothesis if in a similar context but not this specific context), if yes, then not novel 

1 agent to propose experiments



Use generalized context but add one specific sentence to differentiate:

For fact summarization: "In this instance, your task is to provide a factual summary of the known facts based on given information. Stick to well-established, scientifically accepted information without speculation."

For knowledge graph creation: "In this instance, your task is to create a knowledge graph representing the relationships and interactions between the given entities, focusing on functional or physical associations, regulatory relationships, and pathway involvements in the given biological context."

For hypothesis generation: "In this instance, your task is to generate creative yet scientifically plausible hypotheses about potential new functions for the given genes. While your ideas should be grounded in biological principles, you're encouraged to propose novel connections and functions."



In [1]:
import sys
import os

# Add the parent directory of the current script to the Python path
cwd = os.getcwd()
dirname = os.path.dirname(cwd)
print(cwd)
print(dirname)
sys.path.append(dirname)

print(sys.path)

/Users/clara.h/Documents/git_src/agent_evaluation/notebooks
/Users/clara.h/Documents/git_src/agent_evaluation
['/Users/clara.h/Documents/git_src/agent_evaluation/notebooks', '/Users/clara.h/anaconda3/envs/agent_eval/lib/python311.zip', '/Users/clara.h/anaconda3/envs/agent_eval/lib/python3.11', '/Users/clara.h/anaconda3/envs/agent_eval/lib/python3.11/lib-dynload', '', '/Users/clara.h/anaconda3/envs/agent_eval/lib/python3.11/site-packages', '/Users/clara.h/anaconda3/envs/agent_eval/lib/python3.11/site-packages/setuptools/_vendor', '/Users/clara.h/Documents/git_src/agent_evaluation']


In [6]:
# Load database

from models.analysis_plan import AnalysisPlan
from services.analysisrunner import AnalysisRunner
from app.sqlite_database import SqliteDatabase
from app.config import load_database_uri
%reload_ext autoreload
%autoreload 2

database_uri = load_database_uri()
db = SqliteDatabase(database_uri)

### check available LLMs

In [7]:
llm_specs = db.find("llm")
llm_mappings = {}
for llm_spec in llm_specs:
    llm_id = llm_spec["object_id"]
    llm_properties = llm_spec["properties"]
    # print(llm_properties)       
    # only for temperature = 0
    if llm_properties["temperature"] != '0':
        continue
    llm_name = llm_properties["name"]
    llm_mappings[llm_name] = llm_id

llm_mappings

{'GPT_4o_t0': 'llm_e7742ff2-83ac-4dd7-b7e7-cedb0a0dd2cd',
 'Claude3.5_sonnet_t0': 'llm_ab57131f-b207-4583-86a6-e33b8644a189'}

## the gene set function summarization agent 

In [18]:
def templateIO(file):
    with open(file, "r") as f:
        return f.read()

# load the multiagent context file 
context = templateIO("../prompts/contexts/multiagent_context.txt") 
context

"You are an advanced researcher specializing in molecular biology, part of a multi-agent system designed to analyze scientific information. Your ultimate goal is to contribute to the development of plausible, actionable hypotheses for research and potential therapies. Your specific task will vary and may include summarizing facts, verifying information, creating knowledge graphs, or generating hypotheses. Always base your responses on well-established scientific knowledge and peer-reviewed research. Clearly distinguish between verified facts and theoretical concepts. If information is uncertain or speculative, explicitly state so. Maintain scientific rigor in all tasks. If presented with nonsensical, non-scientific, or non-factual questions, respond with \\'Unknown\\'."

In [49]:
# create the agents 
from glob import glob


prompt_template_file = '../prompts/analysts/gene_set_summarizer.txt'
prompt_template = templateIO(prompt_template_file)
# create analyst_specs
analyst_specs = {}

context = templateIO("../prompts/contexts/multiagent_context.txt") 
analyst_name = "gene_set_summarizer1" #updated the summarizer prompt examples, so make new agents
# analyst_name = " ".join(analyst_name)

# updating existing agents with new prompts

for llm in llm_mappings.keys(): # just use Claude3.5 and GPT4o at t = 0 for deterministic results
    # print(analyst_name, llm)
    name = f"{analyst_name}_{llm}"
    analyst_specs[name] = {
        "context": context,
        "prompt_template": prompt_template,
        "llm_id": llm_mappings[llm],
        "description": "Multi agent system -" + analyst_name + "- using " + llm
    }

analyst_specs


{'gene_set_summarizer1_GPT_4o_t0': {'context': "You are an advanced researcher specializing in molecular biology, part of a multi-agent system designed to analyze scientific information. Your ultimate goal is to contribute to the development of plausible, actionable hypotheses for research and potential therapies. Your specific task will vary and may include summarizing facts, verifying information, creating knowledge graphs, or generating hypotheses. Always base your responses on well-established scientific knowledge and peer-reviewed research. Clearly distinguish between verified facts and theoretical concepts. If information is uncertain or speculative, explicitly state so. Maintain scientific rigor in all tasks. If presented with nonsensical, non-scientific, or non-factual questions, respond with \\'Unknown\\'.",
  'prompt_template': '<data>\n{data}\n</data>\n\n<experiment_description>\nThe genes/proteins that were identified as described:\n{experiment_description}\n</experiment_de

In [50]:
Update_existing_agents = True

In [51]:
from models.agent import Agent

existing_agents = db.find("agent")
existing_agent_names = [agent["properties"]["name"] for agent in existing_agents]
analysts = {}
# existing_agent_names 
if Update_existing_agents:
    for name, spec in analyst_specs.items():
        
        if name in existing_agent_names:
            print(f"Updating agent: {name}")
            agent_id = [agent["object_id"] for agent in existing_agents if agent["properties"]["name"] == name][0]
            agent = Agent.load(db, agent_id)
            # print(vars(agent))  
            agent.update(**spec)
            analysts[name] = agent
        else:
            print(f"Creating new agent: {name}")
            analyst = Agent.create(db, spec["llm_id"], spec["context"], spec["prompt_template"], name=name, description=spec.get('description'))
            analysts[name] = analyst
else:
    for name, spec in analyst_specs.items():
        print(f"Creating new agent: {name}")
        analyst = Agent.create(db, 
                    spec["llm_id"], 
                    spec["context"],
                    spec["prompt_template"],
                    name=name,
                    description=spec.get('description'))
        analysts[name] = analyst

Updating agent: gene_set_summarizer1_GPT_4o_t0
Updating agent: gene_set_summarizer1_Claude3.5_sonnet_t0


In [52]:

for agent in analysts.values():
    agent_ids.append(agent.object_id)

agent_ids

['agent_2d22abf4-2a8d-4791-98fd-9c4dbfbfb091',
 'agent_1f0838f5-6e49-4344-89ee-6cbbc3c51a6a',
 'agent_2d22abf4-2a8d-4791-98fd-9c4dbfbfb091',
 'agent_1f0838f5-6e49-4344-89ee-6cbbc3c51a6a']

In [47]:
dataset_id = 'dataset_9cf93b1a-c7af-4084-a5d7-a6d668995247' # just dengue 24hr upregulated proteins,

a_plan_name = "gene_set_summarizer_w_upreg_proteins_24hr"

analysis_plan = AnalysisPlan.create(db, a_plan_name, 
                                    agent_ids, 
                                    dataset_id, 
                                    n_hypotheses_per_agent='1', 
                                    description=
                                    '''gene set summarizer using upregulated proteins at 24hr timepoint
                                    '''
                                    )
print(analysis_plan.object_id)
print(vars(analysis_plan)) 
# 

analysis_plan_1bd74da5-bc5e-4cca-9434-9f7680ac0280
{'db': <app.sqlite_database.SqliteDatabase object at 0x1068ea510>, 'name': 'gene_set_summarizer_w_upreg_proteins_24hr', 'agent_ids': ['agent_2d22abf4-2a8d-4791-98fd-9c4dbfbfb091', 'agent_1f0838f5-6e49-4344-89ee-6cbbc3c51a6a'], 'dataset_id': 'dataset_9cf93b1a-c7af-4084-a5d7-a6d668995247', 'n_hypotheses_per_agent': '1', 'biological_context': 'gene set summarizer using upregulated proteins at 24hr timepoint\n                                    ', 'description': None, 'object_id': 'analysis_plan_1bd74da5-bc5e-4cca-9434-9f7680ac0280', 'created': {'name': 'gene_set_summarizer_w_upreg_proteins_24hr', 'agent_ids': ['agent_2d22abf4-2a8d-4791-98fd-9c4dbfbfb091', 'agent_1f0838f5-6e49-4344-89ee-6cbbc3c51a6a'], 'dataset_id': 'dataset_9cf93b1a-c7af-4084-a5d7-a6d668995247', 'n_hypotheses_per_agent': '1', 'description': 'gene set summarizer using upregulated proteins at 24hr timepoint\n                                    ', 'created': '09.09.2024 13:4

In [53]:

analysis_run = analysis_plan.generate_analysis_run(biological_context="Dengue Virus infection seeking for direct host-based therapeutic targets to inhibit dengue virus replication.")
print(analysis_run.object_id)
print(vars(analysis_run))

runner = AnalysisRunner(db, analysis_run.object_id)
result = runner.run()
print(result)

analysis_run_4422911f-301f-4cfb-9e29-4c5c7e5429b6
{'db': <app.sqlite_database.SqliteDatabase object at 0x1068ea510>, 'analysis_plan_id': 'analysis_plan_1bd74da5-bc5e-4cca-9434-9f7680ac0280', 'agent_ids': ['agent_2d22abf4-2a8d-4791-98fd-9c4dbfbfb091', 'agent_1f0838f5-6e49-4344-89ee-6cbbc3c51a6a', 'agent_2d22abf4-2a8d-4791-98fd-9c4dbfbfb091', 'agent_1f0838f5-6e49-4344-89ee-6cbbc3c51a6a'], 'dataset_id': 'dataset_9cf93b1a-c7af-4084-a5d7-a6d668995247', 'n_hypotheses_per_agent': '1', 'hypothesis_ids': [], 'biological_context': 'Dengue Virus infection seeking for direct host-based therapeutic targets to inhibit dengue virus replication.', 'description': None, 'run_log': '', 'attempts': {'agent_2d22abf4-2a8d-4791-98fd-9c4dbfbfb091': [], 'agent_1f0838f5-6e49-4344-89ee-6cbbc3c51a6a': []}, 'status': 'pending', 'object_id': 'analysis_run_4422911f-301f-4cfb-9e29-4c5c7e5429b6', 'name': 'none', 'user_ids': {'analysis_plan_id': 'analysis_plan_1bd74da5-bc5e-4cca-9434-9f7680ac0280', 'agent_ids': ['agent

## fact checker

In [8]:
from models.analysis_run import AnalysisRun
# load the analysis run 
analysis_run = AnalysisRun.load(db, "analysis_run_4422911f-301f-4cfb-9e29-4c5c7e5429b6")
hypothesis = analysis_run.hypothesis_ids
agent = analysis_run.agent_ids

print(hypothesis, agent)


['hypothesis_af2039a4-fd9d-4902-8d4f-8a84636e95f8', 'hypothesis_7c1beae8-a192-46f5-9d3f-3ab81752a8b2'] ['agent_2d22abf4-2a8d-4791-98fd-9c4dbfbfb091', 'agent_1f0838f5-6e49-4344-89ee-6cbbc3c51a6a', 'agent_2d22abf4-2a8d-4791-98fd-9c4dbfbfb091', 'agent_1f0838f5-6e49-4344-89ee-6cbbc3c51a6a']


In [9]:
from models.hypothesis import Hypothesis
hypothesis_text = Hypothesis.load(db, hypothesis[1]).hypothesis_text # I like the sonnet version better 
print(hypothesis_text)

{
  "Interferon-Stimulated Gene (ISG) Antiviral Response": {
    "proteins": ["IFIH1", "IFIT1", "IFIT2", "IFIT3", "MX1", "MX2", "OAS1", "RIGI"],
    "analysis": "These proteins are part of the interferon-stimulated gene (ISG) response, which is crucial in the innate immune defense against viral infections, including dengue virus. IFIH1 (also known as MDA5) and RIGI (RIG-I) are cytoplasmic pattern recognition receptors that detect viral RNA, triggering the production of type I interferons. The IFIT family proteins (IFIT1, IFIT2, IFIT3) are induced by interferons and have various antiviral functions, including inhibition of viral protein synthesis and sequestration of viral nucleic acids. MX1 and MX2 are GTPases that interfere with viral replication, particularly for RNA viruses. OAS1 is activated by double-stranded RNA and produces 2'-5' oligoadenylates, which activate RNase L to degrade viral RNA. Together, these proteins form a coordinated antiviral response aimed at inhibiting dengue

In [10]:
from models.hypothesis import Hypothesis
import json
import re

def parse_hypothesis(text):
    # Use regex to extract the JSON part of the text
    json_match = re.search(r'{.*}', text, re.DOTALL)
    
    if not json_match:
        raise ValueError("No JSON structure found in the provided text.")
    
    # Extract the JSON string
    json_str = json_match.group(0)
    
    # Parse the JSON string into a Python dictionary
    try:
        parsed_data = json.loads(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding JSON: {e}")
    
    return parsed_data

hypothesis_text = Hypothesis.load(db, hypothesis[1]).hypothesis_text

# Parse the hypothesis data
parsed_hypothesis = parse_hypothesis(hypothesis_text)

# Output the parsed JSON data
print(parsed_hypothesis)

{'Interferon-Stimulated Gene (ISG) Antiviral Response': {'proteins': ['IFIH1', 'IFIT1', 'IFIT2', 'IFIT3', 'MX1', 'MX2', 'OAS1', 'RIGI'], 'analysis': "These proteins are part of the interferon-stimulated gene (ISG) response, which is crucial in the innate immune defense against viral infections, including dengue virus. IFIH1 (also known as MDA5) and RIGI (RIG-I) are cytoplasmic pattern recognition receptors that detect viral RNA, triggering the production of type I interferons. The IFIT family proteins (IFIT1, IFIT2, IFIT3) are induced by interferons and have various antiviral functions, including inhibition of viral protein synthesis and sequestration of viral nucleic acids. MX1 and MX2 are GTPases that interfere with viral replication, particularly for RNA viruses. OAS1 is activated by double-stranded RNA and produces 2'-5' oligoadenylates, which activate RNase L to degrade viral RNA. Together, these proteins form a coordinated antiviral response aimed at inhibiting dengue virus repli

In [12]:
# get proteins 
for key, value in parsed_hypothesis.items():
    print(key)
    proteins = value["proteins"]
    print(proteins)   
    analysis = value["analysis"]
    print(analysis)

Interferon-Stimulated Gene (ISG) Antiviral Response
['IFIH1', 'IFIT1', 'IFIT2', 'IFIT3', 'MX1', 'MX2', 'OAS1', 'RIGI']
These proteins are part of the interferon-stimulated gene (ISG) response, which is crucial in the innate immune defense against viral infections, including dengue virus. IFIH1 (also known as MDA5) and RIGI (RIG-I) are cytoplasmic pattern recognition receptors that detect viral RNA, triggering the production of type I interferons. The IFIT family proteins (IFIT1, IFIT2, IFIT3) are induced by interferons and have various antiviral functions, including inhibition of viral protein synthesis and sequestration of viral nucleic acids. MX1 and MX2 are GTPases that interfere with viral replication, particularly for RNA viruses. OAS1 is activated by double-stranded RNA and produces 2'-5' oligoadenylates, which activate RNase L to degrade viral RNA. Together, these proteins form a coordinated antiviral response aimed at inhibiting dengue virus replication at multiple stages of it

In [14]:
from models.agent import Agent
Agent.load(db, agent[1]).name

'gene_set_summarizer1_GPT_4o_t0'

In [15]:
from services.reference_checker import get_functions_from_paragraph

%reload_ext autoreload
%autoreload 2

functions = get_functions_from_paragraph(db, parsed_hypothesis[list(parsed_hypothesis.keys())[1]], agent[1], verbose=True)

Query:  
I would like to search PubMed to find supporting evidence for the statements in a paragraph. Extract 3-5 keywords from the given paragraph, focusing specifically on gene/protein functions, biological processes, or pathways. 

Example paragraph:  Involvement of pattern recognition receptors: TLR1, TLR2, and TLR3 are part of the Toll-like receptor family, which recognize pathogen-associated molecular patterns and initiate innate immune responses. NOD2 and NLRP3 are intracellular sensors that also contribute to immune activation.
Example response: pattern recognition, receptors, intracellular, Toll-like receptor

Follow these guidelines:

1. Prioritize high-level terms that describe the overall function or role of the genes/proteins mentioned.
2. Include key biological processes or pathways that are central to the paragraph's theme.
3. Consider important molecular targets or interactions mentioned (e.g., viral components).
4. Avoid overly specific molecular mechanisms or individu

In [16]:
functions

['intracellular nucleotide pools',
 'nucleotide metabolism',
 'antiviral response',
 'mitochondrial function',
 'viral replication']

In [17]:
from services.reference_checker import get_references_for_paragraph

%reload_ext autoreload
%autoreload 2

email = 'mhu@health.ucsd.edu'
ref_dict = get_references_for_paragraph(db, parsed_hypothesis[list(parsed_hypothesis.keys())[1]], agent[1], email, n=5, papers_query=20, verbose=True)

Extracting keywords from paragraph
Paragraph:
CMPK2 and NT5C3A are involved in nucleotide metabolism, which can play a role in the antiviral response. CMPK2 is a mitochondrial nucleotide monophosphate kinase that is induced by interferons and may contribute to mitochondrial DNA depletion during the antiviral response. NT5C3A is a pyrimidine 5'-nucleotidase that can modulate intracellular nucleotide pools. Changes in nucleotide metabolism can affect viral replication, as viruses rely on host cell nucleotides for their own genome replication. The upregulation of these enzymes may represent a host strategy to alter nucleotide availability and potentially inhibit dengue virus replication.
Query:  
I would like to search PubMed to find supporting evidence for the statements in a paragraph. Extract 3-5 keywords from the given paragraph, focusing specifically on gene/protein functions, biological processes, or pathways. 

Example paragraph:  Involvement of pattern recognition receptors: TLR1,

TypeError: unhashable type: 'DictionaryElement'

In [76]:
ref_dict

{'paragraph': "These proteins are part of the interferon-stimulated gene (ISG) response, which is crucial in the innate immune defense against viral infections, including dengue virus. IFIH1 (also known as MDA5) and RIGI (RIG-I) are cytoplasmic pattern recognition receptors that detect viral RNA, triggering the production of type I interferons. The IFIT family proteins (IFIT1, IFIT2, IFIT3) are induced by interferons and have various antiviral functions, including inhibition of viral protein synthesis and sequestration of viral nucleic acids. MX1 and MX2 are GTPases that interfere with viral replication, particularly for RNA viruses. OAS1 is activated by double-stranded RNA and produces 2'-5' oligoadenylates, which activate RNase L to degrade viral RNA. Together, these proteins form a coordinated antiviral response aimed at inhibiting dengue virus replication at multiple stages of its lifecycle.",
 'keyword': '(IFIH1[Title/Abstract] OR IFIT1[Title/Abstract] OR IFIT2[Title/Abstract] OR 

In [79]:
import json
import urllib.request

def get_all_gene_symbols(gene_symbol):
    url = f"https://rest.genenames.org/fetch/symbol/{gene_symbol}"
    headers = {"Accept": "application/json"}
    
    request = urllib.request.Request(url, headers=headers)
    try:
        with urllib.request.urlopen(request) as response:
            result = json.loads(response.read().decode())
            # Extract relevant information from the response
            if 'response' in result and 'docs' in result['response'] and len(result['response']['docs']) > 0:
                gene_info = result['response']['docs'][0]
                prev_symbols = gene_info.get('prev_symbol', [])
                alias_symbols = gene_info.get('alias_symbol', [])
                return prev_symbols, alias_symbols  # Return both previous and alias symbols
            else:
                return None, None  # No symbols found
    except Exception as e:
        print("Error detail: ", e)
        return None, None

# Example usage
prev_symbols, alias_symbols = get_all_gene_symbols("IFIH1")
print("Previous Symbols:", prev_symbols)  # Output: Previous symbols
print("Alias Symbols:", alias_symbols)      # Output: Alias symbols

Previous Symbols: []
Alias Symbols: ['MDA-5', 'Hlcd', 'MDA5', 'IDDM19']


In [ ]:
references = ref_dict['references']

In [ ]:
j = 1
formated_citations = ''
formated_paragraph = ''
# sentence list 
paragraph = parsed_hypothesis['Interferon-Stimulated Gene (ISG) Antiviral Response']['analysis']
sentence_list = paragraph.split('.')
# Create a mapping of sentence index to citations
citation_map = {i: [] for i in range(len(sentence_list))}
# format the citations and the paragraph

for reference in references:
    citation = reference['citation']
    support_indexes = reference['support_indexes']
    for index in support_indexes:
        citation_map[index].append(f"[{j}] {citation}")  # Map citation to the corresponding sentence index
    j += 1

# Build the formatted paragraph
for i, sentence in enumerate(sentence_list):
    sentence = sentence.strip()  # Clean up whitespace
    if citation_map[i]:  # If there are citations for this sentence
        citations = ' '.join(citation_map[i])  # Join all citations for this sentence
        formated_paragraph += f"{sentence} {citations}\n"  # Append the sentence with its citations
    else:
        formated_paragraph += f"{sentence}\n"  # Append the sentence without citations

        
        
    j+=1

In [60]:
ref_dict

{'paragraph': 'These proteins are key components of the interferon-induced antiviral response. IFIH1 (also known as MDA5) and RIGI (RIG-I) are pattern recognition receptors that detect viral RNA. The IFIT family proteins (IFIT1, IFIT2, IFIT3) inhibit viral replication by binding to viral RNA or proteins. MX1 and MX2 are GTPases that interfere with viral replication. OAS1 activates RNase L, which degrades viral RNA.',
 'keyword': '(IFIH1[Title/Abstract] OR IFIT1[Title/Abstract] OR IFIT2[Title/Abstract] OR IFIT3[Title/Abstract] OR MX1[Title/Abstract] OR MX2[Title/Abstract] OR OAS1[Title/Abstract] OR RIGI[Title/Abstract]) AND (antiviral[Title/Abstract] OR interferon[Title/Abstract] OR replication[Title/Abstract])',
 'references': [{'citation': 'Yang, Yiying, Song, Jie, Zhao, Hongjun, Zhang, Huali, Guo, Muyao. "Patients with dermatomyositis shared partially similar transcriptome signature with COVID-19 infection." Autoimmunity, 2023, pp. 2220984, doi: https://doi.org/10.1080/08916934.2023.

In [35]:
from services.reference_checker import get_references_for_paragraphs
%reload_ext autoreload
%autoreload 2


hypothesis_id = hypothesis
agent_id = agent # because I only use the context for this purpose so it doesn't matter which agent I use
dict, stored_refs = get_references_for_paragraphs(db, hypothesis_id, agent_id, n=5, papers_query=20, verbose=True)

Extracting keywords from paragraph
Paragraph:
These proteins are key components of the interferon-induced antiviral response. IFIH1 (also known as MDA5) and RIGI (RIG-I) are pattern recognition receptors that detect viral RNA. The IFIT family proteins (IFIT1, IFIT2, IFIT3) inhibit viral replication by binding to viral RNA or proteins. MX1 and MX2 are GTPases that interfere with viral replication. OAS1 activates RNase L, which degrades viral RNA.
Query:  
I would like to search PubMed to find supporting evidence for the statements in a paragraph. Give me a maximum of 3 keywords related to the functions or biological processes in the statements. 

Example paragraph:  Involvement of pattern recognition receptors: TLR1, TLR2, and TLR3 are part of the Toll-like receptor family, which recognize pathogen-associated molecular patterns and initiate innate immune responses. NOD2 and NLRP3 are intracellular sensors that also contribute to immune activation.
Example response: immune response,recep

KeyboardInterrupt: 

In [24]:
dict

{'Interferon-induced antiviral response': {'proteins': ['IFIH1',
   'IFIT1',
   'IFIT2',
   'IFIT3',
   'MX1',
   'MX2',
   'OAS1',
   'RIGI'],
  'analysis': 'These proteins are key components of the interferon-induced antiviral response. IFIH1 (also known as MDA5) and RIGI (RIG-I) are pattern recognition receptors that detect viral RNA. The IFIT family proteins (IFIT1, IFIT2, IFIT3) inhibit viral replication by binding to viral RNA or proteins. MX1 and MX2 are GTPases that interfere with viral replication. OAS1 activates RNase L, which degrades viral RNA.',
  'citations': '[1] Assou, Said, Ahmed, Engi, Morichon, Lisa, Nasri, Amel, Foisset, Florent, Bourdais, Carine, Gros, Nathalie, Tieo, Sonia, Petit, Aurelie, Vachier, Isabelle, Muriaux, Delphine, Bourdin, Arnaud, De Vos, John. "The Transcriptome Landscape of the In Vitro Human Airway Epithelium Response to SARS-CoV-2." International journal of molecular sciences, 2023, pp.  , doi: https://doi.org/10.3390/ijms241512017\n[2] Fleith, Re